In [ ]:
# SDMDC Controller Code Integrating LSTM Local IDS, CCTT Global IDS, and Lemurs Optimizer

import torch
import os
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# Importing the Local and Global IDS Models
import sys
import importlib.util

lstm_dir = "/kaggle/input/local-ids-using-lstm"
lstm_files = [f for f in os.listdir(lstm_dir) if f.endswith(".py")]
if lstm_files:
    lstm_spec = importlib.util.spec_from_file_location("lstm_local_ids", os.path.join(lstm_dir, lstm_files[0]))
    lstm_module = importlib.util.module_from_spec(lstm_spec)
    lstm_spec.loader.exec_module(lstm_module)
    LSTMIDS = lstm_module.LSTMIDS
else:
    raise FileNotFoundError(f"No Python files found in {lstm_dir}")
cctt_dir = "/kaggle/input/cctt-with-lo/"
cctt_files = [f for f in os.listdir(cctt_dir) if f.endswith(".py")]
if cctt_files:
    cctt_spec = importlib.util.spec_from_file_location("cctt_with_lo", os.path.join(cctt_dir, cctt_files[0]))
    cctt_module = importlib.util.module_from_spec(cctt_spec)
    cctt_spec.loader.exec_module(cctt_module)
    CCTTWithLO = cctt_module.CCTTWithLO
else:
    raise FileNotFoundError(f"No Python files found in {cctt_dir}")

class SoftwareDefinedMulticloudDefenseController:
    def __init__(self, cloud_ids, input_dim=64, seq_len=10):
        self.cloud_ids = cloud_ids
        self.seq_len = seq_len
        self.input_dim = input_dim

        self.local_ids = {cid: LSTMIDS(input_dim, 128, 1) for cid in cloud_ids}
        self.global_ids = CCTTWithLO(input_dim=input_dim, seq_len=seq_len)

        self.traffic_logs = {cid: [] for cid in cloud_ids}
        self.policy_db = {}

    def ingest_traffic(self, cloud_id, traffic_tensor):
        self.traffic_logs[cloud_id].append(traffic_tensor)

    def run_local_ids(self):
        local_scores = {}
        for cid, model in self.local_ids.items():
            if len(self.traffic_logs[cid]) >= self.seq_len:
                input_seq = torch.stack(self.traffic_logs[cid][-self.seq_len:]).unsqueeze(0)
                score = model(input_seq)
                local_scores[cid] = score.item()
            else:
                local_scores[cid] = 0.0
        return local_scores

    def aggregate_global_input(self):
        sequence_list = []
        for cid in self.cloud_ids:
            sequence = torch.stack(self.traffic_logs[cid][-self.seq_len:])
            sequence_list.append(sequence)
        return torch.stack(sequence_list).unsqueeze(0)

    def detect_global_threats(self):
        input_tensor = self.aggregate_global_input()
        threat_scores = self.global_ids(input_tensor)
        return threat_scores.squeeze(0).tolist()

    def enforce_policy(self, threat_scores):
        for i, score in enumerate(threat_scores):
            if score > 0.6:
                self.policy_db[self.cloud_ids[i]] = {
                    'action': 'block',
                    'score': score,
                    'reason': 'Global IDS - anomaly detected'
                }

    def run_cycle(self):
        local_scores = self.run_local_ids()
        global_scores = self.detect_global_threats()
        self.enforce_policy(global_scores)
        return local_scores, global_scores, self.policy_db

    def evaluate_results(self, y_true, y_scores):
        y_pred = [1 if score > 0.5 else 0 for score in y_scores]
        cm = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='macro')
        recall = recall_score(y_true, y_pred, average='macro')
        f1 = f1_score(y_true, y_pred, average='macro')
        roc = roc_auc_score(y_true, y_scores)

        print("Confusion Matrix:\n", cm)
        print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f} | ROC-AUC: {roc:.4f}")

        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.title("Confusion Matrix")
        plt.imshow(cm, cmap='Blues')
        plt.colorbar()

        plt.subplot(1, 2, 2)
        plt.title("Threat Detection Scores")
        plt.plot(y_scores, label='Predicted Scores')
        plt.plot(y_true, label='True Labels', linestyle='--')
        plt.legend()
        plt.tight_layout()
        plt.show()


# Preprocessing Bot-IoT Dataset

def preprocess_bot_iot(csv_path, n_features=64, n_samples=100):
    df = pd.read_csv(csv_path)
    df = df.select_dtypes(include=[np.number])
    df = df.fillna(0).iloc[:n_samples, :n_features]
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df.values)
    return torch.tensor(scaled, dtype=torch.float32)


# Run the SDMDC Pipeline

def simulate_run():
    cloud_ids = ['aws', 'azure', 'gcp']
    controller = SoftwareDefinedMulticloudDefenseController(cloud_ids)

    data_tensor = torch.cat([preprocess_bot_iot(f"/kaggle/input/bot-iot/{file}") for file in sorted(os.listdir("/kaggle/input/bot-iot")) if file.endswith('.csv')])

    for t in range(10):
        for i, cid in enumerate(cloud_ids):
            controller.ingest_traffic(cid, data_tensor[t + i])

    local_scores, global_scores, policies = controller.run_cycle()

    print("Local IDS Outputs:", local_scores)
    print("Global IDS Threat Scores:", global_scores)
    print("Policies Enforced:", policies)

    # Simulated ground truth for evaluation (e.g., [Benign, Attack, Benign])
    y_true = [0, 1, 0]
    controller.evaluate_results(y_true, global_scores)


if __name__ == '__main__':
    simulate_run()
